We re-create numerical artefacts in the implementation of GPR with a periodic kernel for forward prediction. 
We consider a simple truth trajectory given by a deterministic sine curve. Next, we configure the GPRP algorithm and plot predictions. 

To get started, I import some python software I will need:

In [1]:
import sys
sys.path.append('../../../../../')
from plot_tools.fig_preamble import * # imports numpy, matplotlib, Py2PDF, plot_helper_funcs, plot_figstyle_sheet
import GPy


from gpr.common import get_data, simple_unlearnable_sine
from data_tools.load_raw_cluster_data import LoadExperiment as le
from plot_tools.plot_figstyle_sheet import color_pallete, predmarker, COLOURDICT, STYLEDICT
from analysis_tools.common import sqr_err

/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1206: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markerfacecolor != fc:
/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1182: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markeredgecolor != ec:


I also set up filepaths for saving data

In [2]:
ver=0
datapath = '../../../../DATA_v'+str(ver)+'_/' #v'+str(ver)+'/DATA_v'+str(ver)+'/'
savefig2 = '../svg/paper_v0_'
figdata = '../svg/fig_data/paper_v0_'
figname2 = 'GPR_new_BR'

Now, let's consider how to configure the GPRP algorithm with limited apriori information about the truth.
In our application, let $\Delta t$ represent the shortest possible time-spacing in our procedure between two sequential measurements from an experiment, and let $\Delta t N_T$ represent the longest possible time spacing observed in the measurement record. 

We use physical arguments to link the periodicity and the lengthscale to the longest and shortest timescales of the observed system during training. In configuring a GPR algorithm, one decides or optimises $\omega_0^{(B)}$. For the physically set paramter, $\Delta t$ in our procedure, a classical analysis suggests that one needs $2 \pi /(\Delta t \omega_0^{(B)})$ for a computational method to resolve Fourier components of true noise to an order $\omega_0^{(B)}$. Further, any changes in the continuous process below $\Delta t$ will not be observed. These reasons lead us to propose the following initial conditions for kernel hyperparameters at the outset of a procedure:, namely:

    - p = N_train
    - l = 3 * Delta T 
    


In [3]:
# Set experimental parameters
n_train = 2000
Delta_T = 0.001
n_predict = 100
n_testbefore=50

In [4]:
# Set initial starting values for L-BFGS-B OPtimiser in GPy
sigma_0 = 1.0
R_0 =  1.0

In [5]:
# Set length scale and periodicity initial values for L-BFGS-B OPtimiser in GPy
length_scale_0 = 3.0*Delta_T
period_0 = 2000.0

true = 9.0/3.0
multipler = 7. # must be an integer
spacing = true / multipler
steps = (1./ spacing)/ 0.001
print steps
print spacing

print basis
print true / (1./(2000*0.001)) # integer

We now implement optimised GPRP model but with constrained periodicities

In [6]:
total_runs = 50
periodicities = ((2000, 2001), (0, 10**3), (10**3, 10**4), (10**4, 10**5), (None, None))

In [7]:
pred_list= np.zeros((len(periodicities), total_runs, n_predict+n_testbefore))
opt_list = np.zeros((len(periodicities), total_runs, 4))
truth_list= np.zeros((len(periodicities), total_runs, n_predict+n_testbefore))
failed_runs_list = np.zeros((len(periodicities), total_runs))

In [ ]:
dataobject = le(19, 5, 
                skip = 1,
                GPRP_load='No', GPRP_path = './',
                LKFFB_load = 'Yes', LKFFB_path = datapath,
                AKF_load='No', AKF_path = './',
                LSF_load = 'No', LSF_path = './')

LKFFB: Data Loaded? Yes


In [ ]:
for idx_run in xrange(total_runs): 

    X, Y, TestX, truth, msmts = get_data(dataobject, points=2000, randomize='y')
  
    for idx_per in xrange(len(periodicities)):        
        
        # Reset
        kernel_per=0
        gauss=0
        exact=0
        m1=0

        
        # Built model
        kernel_per = GPy.kern.StdPeriodic(1, period=period_0, variance=sigma_0, 
                                          lengthscale=length_scale_0)
        gauss = GPy.likelihoods.Gaussian(variance=R_0)
        exact = GPy.inference.latent_function_inference.ExactGaussianInference()
        m1 = GPy.core.GP(X=X, Y=Y, kernel=kernel_per, likelihood=gauss, inference_method=exact)

        
        # Add constraints        
        if idx_per != len(periodicities)-1: # last condition
            print("Peridocity is constrained...")
            m1.std_periodic.period.constrain_bounded(periodicities[idx_per][0], periodicities[idx_per][1])
            

        print("Optimise GPRP for:", idx_per, idx_run)
        print(m1)
        
        try:
            m1.optimize(optimizer=None, messages=False)
            opt_list[idx_per, idx_run, :] = [m1.std_periodic.period[0], m1.std_periodic.lengthscale[0], m1.std_periodic.variance[0], gauss.variance[0]]
            pred_list[idx_per, idx_run, :] = m1.predict(TestX)[0].flatten()
            print("Optimisation complete!")
            print(m1)

        except:
            opt_list[idx_per, idx_run, :] = np.zeros(4)
            pred_list[idx_per, idx_run, :] = np.zeros(int(TestX.shape[0]))
            failed_runs_list[idx_per, idx_run] = idx_run
            print("Unexpected error:", sys.exc_info()[0])
            print("Failed run", idx_run)

        truth_list[idx_per, idx_run, :] = truth[int(TestX[0]):]
        
        np.savez(figdata+figname2,
                 truth_list=truth_list, 
                 pred_list=pred_list,
                 opt_list=opt_list, 
                 failed_runs_list=failed_runs_list)


 ../../../../../gpr/common.py:66: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 ../../../../../gpr/common.py:67: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


Peridocity is constrained...
('Optimise GPRP for:', 0, 0)

Name : gp
Objective : 154958413.504
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |   constraints   |  priors
  std_periodic.variance     |     1.0  |       +ve       |        
  std_periodic.period       |  2000.0  |  2000.0,2001.0  |        
  std_periodic.lengthscale  |   0.003  |       +ve       |        
  Gaussian_noise.variance   |     1.0  |       +ve       |        
Optimisation complete!

Name : gp
Objective : 14242.2469152
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |            value  |   constraints   |  priors
  std_periodic.variance     |    862783.756191  |       +ve       |        
  std_periodic.period       |           2000.0  |  2000.0,2001.0  |        
  std_periodic.lengthscale  |  0.0137623756107  |       +ve       |        
  Gaussian_noise.variance 

 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:113: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:132: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:134: RuntimeWarning:divide by zero encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:134: RuntimeWarning:invalid value encountered in multiply
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:136: RuntimeWarning:divide by zero encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:136: RuntimeWarning:invalid value encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:144: RuntimeWarning:invalid value encountered in multi

Optimisation complete!

Name : gp
Objective : 367722479.278
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |            value  |  constraints  |  priors
  std_periodic.variance     |    1.00557320301  |      +ve      |        
  std_periodic.period       |    285.594992872  |  0.0,1000.0   |        
  std_periodic.lengthscale  |  0.0030136602204  |      +ve      |        
  Gaussian_noise.variance   |    1.04651621674  |      +ve      |        
Peridocity is constrained...
('Optimise GPRP for:', 2, 0)

Name : gp
Objective : 154958413.504
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |   constraints    |  priors
  std_periodic.variance     |     1.0  |       +ve        |        
  std_periodic.period       |  2000.0  |  1000.0,10000.0  |        
  std_periodic.lengthscale  |   0.003  |       +ve        |        
  Gaussian_noise.varian

 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:136: RuntimeWarning:overflow encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:106: RuntimeWarning:invalid value encountered in greater
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:111: RuntimeWarning:invalid value encountered in greater


('Unexpected error:', <class 'numpy.linalg.linalg.LinAlgError'>)
('Failed run', 0)
('Optimise GPRP for:', 4, 0)

Name : gp
Objective : 154958413.504
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |  constraints  |  priors
  std_periodic.variance     |     1.0  |      +ve      |        
  std_periodic.period       |  2000.0  |      +ve      |        
  std_periodic.lengthscale  |   0.003  |      +ve      |        
  Gaussian_noise.variance   |     1.0  |      +ve      |        


 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:109: RuntimeWarning:overflow encountered in expm1


Optimisation complete!

Name : gp
Objective : 14773.6411401
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |            value  |  constraints  |  priors
  std_periodic.variance     |    118798.714407  |      +ve      |        
  std_periodic.period       |    1959.62750846  |      +ve      |        
  std_periodic.lengthscale  |  0.0106766672119  |      +ve      |        
  Gaussian_noise.variance   |    120207.858363  |      +ve      |        
Peridocity is constrained...
('Optimise GPRP for:', 0, 1)

Name : gp
Objective : 149537098.044
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |   constraints   |  priors
  std_periodic.variance     |     1.0  |       +ve       |        
  std_periodic.period       |  2000.0  |  2000.0,2001.0  |        
  std_periodic.lengthscale  |   0.003  |       +ve       |        
  Gaussian_noise.variance  

In [ ]:
forecastng_errors = np.zeros(( len(periodicities), total_runs, n_predict ))
bayes_pred_risk = np.zeros(( len(periodicities), n_predict ))

prd_zero = np.mean(truth_list**2, axis=1)[:, n_testbefore:]


plt.figure()
labels = ['k=0', 'k=70','k=10^2', 'k=10^4', 'k=10^6']


for idx_per in [0,1, 2, 3, 4]:#xrange(len(periodicities)):
    
    for case in xrange(total_runs):
        
        forecastng_errors[idx_per, case, :] = sqr_err(pred_list[idx_per, case, n_testbefore:], 
                                                      truth_list[idx_per, case, n_testbefore:])
   
    bayes_pred_risk[idx_per, :] = np.mean(forecastng_errors[idx_per, :, :], axis=0) / prd_zero[idx_per]
    
    plt.plot(np.arange(n_predict), bayes_pred_risk[idx_per],'--',  label=idx_per)
    
plt.yscale('log')
plt.xscale('log')
plt.legend(loc=1)
plt.show()

In [ ]:
failed_runs_list